# Code Inspector Agent

A simple example demonstrating how to build an agent that can search through codebases and answer questions about code structure. This agent uses custom actions to search code repositories and analyze findings, allowing developers to quickly understand unfamiliar codebases.

First, index the codebase in a vector store.

Set `repo_dir` to the path of the repository you want to index.

In [1]:


from moatless.index import CodeIndex, IndexSettings
from moatless.repository import FileRepository
from moatless.workspace import Workspace

import os
import logging
from dotenv import load_dotenv

#logging.basicConfig(level=logging.WARNING)
#logging.getLogger("moatless").setLevel(logging.INFO)

load_dotenv()

# An OPENAI_API_KEY is required to use the OpenAI Models
index_settings = IndexSettings(
    # embed_model="text-embedding-3-small",
    embed_model="jinaai/jina-code-embeddings-0.5b",
)


repo_dir = "/home/schz/research/Agents/moatless-tools"
# repo_dir = "/home/schz/research/Agents/moatless-tools/scripts"
persist_dir = "/home/schz/research/Agents/moatless-tools/.index"

file_repo = FileRepository(repo_path=repo_dir)

if os.path.exists(persist_dir):
    print(f"Found existing index on {persist_dir}")
    code_index = CodeIndex.from_persist_dir(persist_dir, file_repo=file_repo)
else:
    code_index = CodeIndex(file_repo=file_repo, settings=index_settings)
    nodes, tokens = code_index.run_ingestion()
    print(f"Indexed {nodes} nodes and {tokens} tokens")
    code_index.persist(persist_dir)

workspace = Workspace(
    repository=file_repo,
    code_index=code_index,
    shadow_mode=True # No changes are persisted to the file system
)

Found existing index on /home/schz/research/Agents/moatless-tools/.index
Loading llama_index.core.storage.kvstore.simple_kvstore from /home/schz/research/Agents/moatless-tools/.index/docstore.json.


The Code Inspector Agent combines a deterministic language model with specialized code analysis actions to create a read-only code exploration tool.

The model uses GPT-4.1-mini with temperature set to 0 for consistent, focused responses, while the prompt guides the agent through a systematic workflow - understanding requests, locating relevant code, gathering details, and reporting findings concisely.

The action toolkit provides all necessary capabilities for thorough code analysis:
* `Think()` enables private reasoning
* `SemanticSearch()` finds relevant code sections using semantic search to the vector store
* `FindClass()` and `FindFunction()` locate specific components
* `FindCodeSnippet()` identifies code patterns
* `ReadFile()` examines file contents
* `Respond()` delivers findings to users

In [2]:
from moatless.actions import ReadFile, FindClass, FindFunction, FindCodeSnippet, StringReplace, CreateFile, Finish, SemanticSearch
from moatless.actions.respond import Respond
from moatless.actions.think import Think
from moatless.completion import ToolCallCompletionModel
from moatless.completion.react import ReActCompletionModel, JsonCompletionModel
from moatless.completion.completion_mixin import CompletionModelMixin
from moatless.flow.loop import AgenticLoop
from moatless.message_history import MessageHistoryGenerator
from moatless.agent import ActionAgent

# completion_model = ToolCallCompletionModel(
completion_model = ReActCompletionModel(
   #  model="gpt-4.1-mini",
    model="ollama/devstral-64k",
    temperature=0.0,
    model_api_key="",
    few_shot_examples=True
)

# extra_completion_model = JsonCompletionModel(
#       model=completion_model.model,
#       temperature=0.0,
#       max_tokens=completion_model.max_tokens,
#       timeout=completion_model.timeout,
#       few_shot_examples=completion_model.few_shot_examples,
#       headers=completion_model.headers,
#       params=completion_model.params,
#       merge_same_role_messages=completion_model.merge_same_role_messages,
#       thoughts_in_action=completion_model.thoughts_in_action,
#       disable_thoughts=completion_model.disable_thoughts,
#       message_cache=completion_model.message_cache,
#       model_base_url=completion_model.model_base_url,
#       model_api_key=completion_model.model_api_key,
# )


system_prompt = """# Code Inspector Agent Prompt

You are a specialized AI agent whose sole purpose is to explore a codebase and report findings. You cannot modify any code or communicate directly with the user—only return information based on built-in inspection functions.

## 1. Internal Reasoning
- Before starting any inspection task or when facing ambiguity, record a private chain of thought—do not expose it to the user.

## 2. Investigation Workflow
1. **Understand Request**  
   Read the user’s query to determine which parts of the codebase to inspect.  
2. **Locate Code**  
   Use code-search functions to find relevant files, classes, functions, or snippets.  
3. **Gather Details**  
   Open and read the identified code to extract requested information (signatures, dependencies, comments, complexity, etc.).  
4. **Report Findings**  
   Summarize the results clearly and concisely, quoting file paths and code excerpts as needed. Use the `Respond` tool to return the findings.

## 3. Guidelines
- **Read-Only**: Do not alter or write any files—only inspect and report.  
- **Precision**: Provide exact file names, line numbers, and code snippets.  
- **Context**: When relevant, include surrounding code or module relationships.  
- **Brevity**: Keep responses focused on the user’s question.  
- **Tool Usage**: Always use tools, respond by using the `Respond` tool.
"""

actions=[
   Think(),
   SemanticSearch(),
   FindClass(),
   FindFunction(),
   FindCodeSnippet(),
   ReadFile(),
   Respond(),
]

agent = ActionAgent(
    completion_model=completion_model,
    system_prompt=system_prompt,
    actions=actions,
    memory=MessageHistoryGenerator(),
)


# for action in agent.actions:
#    if isinstance(action, CompletionModelMixin):
#          action.completion_model = extra_completion_model.clone()


loop = AgenticLoop.create(agent=agent, max_iterations=20)

# last_node = await loop.run("Return a detailed description of the repository implementation and what dependencies it has", workspace=workspace)
# print(last_node.observation.message)s


In [6]:
nodes = n.get_all_nodes()

In [21]:
for k in nodes[-2]:
    print(k)
    


('node_id', 5)
('parent', Node(node_id=4, parent=Node(node_id=3, parent=Node(node_id=2, parent=Node(node_id=1, parent=Node(node_id=0, parent=None, children=[Node(node_id=1, parent=Node(node_id=0, parent=None, children=[...], artifact_changes=[], user_message=None, assistant_message=None, thoughts=None, thinking_blocks=None, action_steps=[], file_context=FileContext(show_code_blocks=False), completions={}, possible_actions=[], is_duplicate=False, terminal=False, error=None, reward=None, selection=None, visits=0, value=None, max_expansions=None, agent_id=None, feedback_data=None, timestamp=datetime.datetime(2025, 9, 15, 23, 38, 42, 58631), discriminator_result=None, evaluation_result=None), children=[Node(node_id=2, parent=Node(node_id=1, parent=Node(node_id=0, parent=None, children=[...], artifact_changes=[], user_message=None, assistant_message=None, thoughts=None, thinking_blocks=None, action_steps=[], file_context=FileContext(show_code_blocks=False), completions={}, possible_actions=

In [3]:
n = await loop.run("Tell me more about moatless/index folder", workspace=workspace)

Initial JSON parse failed, attempting alternate cleanup
Post validation failed with retry message: Invalid format for SemanticSearch. Error: 1 validation error for SemanticSearch
  Value error, query cannot be empty [type=value_error, input_value={'file_pattern': 'moatles...gory': 'implementation'}, input_type=dict]

Expected schema:
Requires a JSON response with the following schema: {"description": "Use this when you don't know exact names or code but want to find related functionality.\n\nPerfect for:\n- Finding functionality by description: query=\"code that handles password hashing\"\n- Finding related test cases: query=\"tests for user registration\", category=\"test\"\n- Finding implementations: query=\"database connection pooling\", category=\"implementation\"\n- Finding patterns: query=\"error handling for API requests\"\n\nThis is the most flexible search when you:\n- Don't know exact function/class names\n- Want to explore how certain features are implemented", "properties":